# Test Model
- test model on holdout sessions

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import seaborn as sns
import plotly.express as px
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from torch.utils.data import TensorDataset,DataLoader
from tqdm import tqdm

In [ ]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Define Model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(300, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits  

model = MLP().to(device)
model.load_state_dict(torch.load('model/model.pt'))
    
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Test by Predicting with Model

# Get one file
index = 53
step = 10000    # for memory

# Read in holdout session
dataset = TensorDataset(torch.load(f'pipeline/datasets/{index}-holdout.pt'))
session_length = len(dataset)
dataloader = DataLoader(dataset, batch_size=step)

model.eval()

preds = []
n_correct = 0
loss = 0

for X, y_true in dataloader:
    X = X.to(device) 
    y_true = y_true.to(device)

    logits = model(X)
    pred = torch.round(nn.Sigmoid()(logits))

    n_correct += sum(y_true == pred)
    preds += pred.flatten().tolist()
    loss += criterion(logits, y_true).item()

preds = np.array(preds).reshape(-1,1)
accuracy = (n_correct / len(y_true)).item()
loss = loss/session_length
print(f'Accuracy: {100*accuracy:.4}%')
print(f'Loss: {loss:.4}')

session_length == y_true

In [ ]:
# Visualize predictions
df = pd.read_csv(f'pipeline/1_xyz/{index}.csv')
df['preds'] = np.pad(preds.flatten()*10, (0, 99), mode='constant', constant_values=0)

figure = px.line(df)
figure.show(renderer='browser')

In [ ]:
# Confusion Matrix 
# TODO use other one

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true.detach().cpu(), preds)
sns.heatmap(cm, annot=True)